# TP3 - Automatic Segmentation of Mails
This Lab aims to build an email segmentation tool, dedicated to separate the email header from its
body. It is proposed to perform this task by learning a HMM (A, B, π) with two states, one (*state 1*) for
the header, the other (*state 2*) for the body. In this model, it is assumed that each mail actually contains
a header : the decoding necessarily begins in the state 1.

Knowing that each mail contains exactly one header and one body, each mail follows once the transition
from 1 to 2.

### Q1 : Give the value of the π vector of the initial probabilities

$$π^T = 
\begin{matrix} 
1 \\ 0 
\end{matrix}
$$

Because the initial probability of being in the state 0 (header) is always true.

### Q2 : What is the probability to move from state 1 to state 2 ? What is the probability to remain in state 2 ? What is the lower/higher probability ? Try to explain why

Given the matrix:

$$ A = \begin{matrix} 
0.999218078035812 & 0.000781921964187974 \\
0 & 1
\end{matrix}$$

The probability of moving from state 1 to state 2 is `P(2|1) = 0.000781921964187974` ; the probability of state 2 is `P(2|2) = 1` . 

It is normal for P(2|2) to be higher, since once a character belonging to the *body* of the mail has been found, all the following observations will belong to the same state : no other *header* is found after the *body* in a mail.

### Q3 : What is the size of the corresponding matrix ?

Because the ASCII characters are 256 (`N = 256`), the size of the corresponding matrix will be `256x2`, where each row represents the discrete probability distribution of the character *c* given the state *s*.

In [1]:
import numpy as np

# Viterbi - test
O = range(0,256)
S = [0,1]
pi = [1,0]
A = [[0.999218078035812, 0.000781921964187974],[0, 1]]
B = np.loadtxt('P.text', dtype=float)

In [2]:
# Implementation of Viterbi algorithm
"""
Implementation of the Viterbi algorithm.
Finds the best segmentation of the text provided, 
and returns the most probable states sequence
Parameters explaination:
:param O: = characters codification (ASCII, range(0,256))
:param S: = possible states (header = 0, body = 1)
:param A: = state transition probability matrix
:param B: = matrix probability of character c being in state s (256x2, contained in P.text)
:param pi: = initial probability vector
:param Y: = observation in the mail (mail.dat)
:return X: = most likely hidden state sequence
"""
def ViterbiAlgorithm(O, S, A, B, pi, Y):
    A = np.matrix(A)
    T = len(Y)
    T1 = np.zeros((len(S), T))
    T2 = np.zeros((len(S), T))
    for s in S:
        T1[s,0] = pi[s]*B[Y[0]][s] # vector of most likely path so far
        T2[s,0] = 0 # most likely path for previous observation
#     print("T1[:, 0] = "+str(T1[:,0]))
    for t in range(1,T):
        for s in S:
#             print("LOGGED T1[:, "+str(t-1)+"] = "+str(np.log(T1[:,t-1])))
#             print("LOGGED A[:][s] = "+str(np.log(A[:,s])))
#             print("B[Y["+str(t)+"],s] = "+str(B[Y[t],s]))
#             result = T1[:, t-1]*A[:,s].reshape(2,1)*B[Y[t],s]
            result = [a*b*B[Y[t],s] for a,b in zip(T1[:, t-1],A[:,s])]
            result += np.finfo(np.double).tiny
#             print("Result = "+str(result))
            T1[s, t] = max(result)
            T2[s, t] = np.argmax(result)
#         print("T1[:, "+str(t)+"] = "+str(T1[:, t]))
    Z = [0 for t in Y]
    X = [0 for t in Y]
    T = T - 1
    Z[T] = int(np.argmax(T1[:,T]))
    X[T] = S[Z[T]]
    for i in range(T, 0, -1):
        Z[i-1] = int(T2[Z[i], i])
        X[i-1] = S[Z[i-1]]
    return X

In [3]:
Y = np.loadtxt('dat/mail11.dat',dtype=int)

X = ViterbiAlgorithm(O, S, A, B, pi, Y)
print("Observation size: "+str(len(Y)))
print("Expected change point in mail11.dat : ~2850")
print("Obtained change point : "+str(np.bincount(X)[0]))

Observation size: 3475
Expected change point in mail11.dat : ~2850
Obtained change point : 161


In [4]:
Y = np.loadtxt('dat/mail30.dat',dtype=int)

X = ViterbiAlgorithm(O, S, A, B, pi, Y)
print("Observation size: "+str(len(Y)))
print("Expected change point in mail30.dat : ~2250")
print("Obtained change point : "+str(np.bincount(X)[0]))

Observation size: 5160
Expected change point in mail30.dat : ~2250
Obtained change point : 180


In [5]:
def printPath(path):
    summary = []
    summary.append({ 'start': 0, 'end': 0, 'value': path[0]})
    for i in range(1, len(path)):
        if path[i] != path[i-1]:
            summary.append({ 'start': i, 'end': i, 'value': path[i] })
        else:
            summary[-1]['end'] = i
    
    for s in summary:
        print('State ' + str(s['value']) + ' from observation ' + str(s['start']) + ' to ' + str(s['end']))
        
for i in range(11,31):
    Y = np.loadtxt('./dat/mail'+str(i)+'.dat', dtype=int)
#     p = viterbiAlgorithm(obs=observations, states=[1,2], start_prob=[1,0], trans=A, cond_prob=distributions)
    X = ViterbiAlgorithm(O, S, A, B, pi, Y)
    print('--- MAIL ' + str(i) + ' STATES : ')
    printPath(X)

--- MAIL 11 STATES : 
State 0 from observation 0 to 160
State 1 from observation 161 to 3474
--- MAIL 12 STATES : 
State 0 from observation 0 to 160
State 1 from observation 161 to 3992
--- MAIL 13 STATES : 
State 0 from observation 0 to 179
State 1 from observation 180 to 3327
--- MAIL 14 STATES : 
State 0 from observation 0 to 136
State 1 from observation 137 to 6575
--- MAIL 15 STATES : 
State 0 from observation 0 to 179
State 1 from observation 180 to 6807
--- MAIL 16 STATES : 
State 0 from observation 0 to 179
State 1 from observation 180 to 2626
--- MAIL 17 STATES : 
State 0 from observation 0 to 192
State 1 from observation 193 to 3424
--- MAIL 18 STATES : 
State 0 from observation 0 to 179
State 1 from observation 180 to 3076
--- MAIL 19 STATES : 
State 0 from observation 0 to 192
State 1 from observation 193 to 2619
--- MAIL 20 STATES : 
State 0 from observation 0 to 179
State 1 from observation 180 to 2433
--- MAIL 21 STATES : 
State 0 from observation 0 to 167
State 1 from o